In [3]:
import tensorflow as tf

# Create TensorFlow object called hello_constant
hello_constant = tf.constant('Hello World!')

with tf.Session() as sess:
    # Run the tf.constant operation in the session
    output = sess.run(hello_constant)
    print(output)

b'Hello World!'


tensorflow stores data as tensors, which is an onject which can store stuff.

constant data is stored as a [tf.constant](http://devdocs.io/tensorflow~python/tf/constant). The output of a constant tensor doesn't change:

In [2]:
# A is a 0-dimensional int32 tensor
A = tf.constant(1234) 
# B is a 1-dimensional int32 tensor
B = tf.constant([123,456,789]) 
 # C is a 2-dimensional int32 tensor
C = tf.constant([ [123,456,789], [222,333,444] ])

A Tensorflow session is an environment for running a graph.

```python
with tf.Session() as sess:
    output = sess.run(hello_constant)```
    
The `sess.run` function runs the hello_constant tensor created above.

Tensorflow can't just take a dataset x as in input, it has to be a tensor. [tf.placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) is used for dynamic data, which is fed into a tensor part at a time. 

In [17]:
x = tf.placeholder(tf.string)
y = tf.placeholder(tf.int32)
z = tf.placeholder(tf.float32)

with tf.Session() as sess:
    output = sess.run(x, feed_dict={x: 'Test String', y: 123, z: 45.67})
    print(output)

Test String


## Tensorflow math

In [30]:
x = tf.add(5, 2)  # 7
y = tf.subtract(10, 4) # 6
z = tf.multiply(2, 5)  # 10
print(x ,y ,z)

Tensor("Add_5:0", shape=(), dtype=int32) Tensor("Sub_7:0", shape=(), dtype=int32) Tensor("Mul_2:0", shape=(), dtype=int32)


math operations have to use tensors of the same type, so watch out for floats, ints, etc. The below fails becuase that tries to subtract an int from a float:
```python
tf.subtract(tf.constant(2.0),tf.constant(1))```

Use [tf.cast](http://devdocs.io/tensorflow~python/tf/cast) to convert a tensor to another type:

In [25]:
tf.subtract(tf.cast(tf.constant(2.0), tf.int32), tf.constant(1))   # 1

<tf.Tensor 'Sub_5:0' shape=() dtype=int32>

to print the value of a tensorflow variable, you have to run it in a session:

In [38]:
x = tf.add(5, 2)  # 7
with tf.Session() as s:
    output = s.run(x)
print(x) # prints out a tensor object
output # prints the output of the tensor x

Tensor("Add_13:0", shape=(), dtype=int32)


7

In [52]:
# TODO: Convert the following to TensorFlow:
x = tf.constant(10)
y = tf.constant(2)
x_minus_y = tf.divide(x,y)
z = tf.subtract(x_minus_y, tf.cast(1.0, tf.float64))

# TODO: Print z from a session
with tf.Session() as sess:
    output = sess.run(z)
    print(output)

4.0


## Classification

this is the central building block of ML.

Logistic classifiers use a linear function like Wx + b = y and a softmax funtion to assign probablities to output.

![](images/logistic_classifier.png)

Softmax can take any kind of scores and turn them into probalities.

> The most common operation in neural networks is calculating the linear combination of inputs, weights, and biases.

> Here, W is a matrix of the weights connecting two layers. The output y, the input x, and the biases b are all vectors.

Both weights and biases need to be modified as the NN trains, so they use a [tf.variable](https://www.tensorflow.org/api_docs/python/tf/Variable) class:


In [54]:
x = tf.Variable(5)

the [tf.Variable](https://www.tensorflow.org/api_docs/python/tf/Variable) class stores its state in the session, so it must be initialized manually. the [tf.global_variables_initializer()](https://www.tensorflow.org/api_docs/python/tf/global_variables_initializer) function initializes all variable tensors.

In [55]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)

We need an initial value for weights and biases, and normally we just use random values from a normal distrubution, for which we can use the [tf.truncated_normal()](https://www.tensorflow.org/api_docs/python/tf/truncated_normal) function.

In [66]:
n_features = 120
n_labels = 5
weights = tf.Variable(tf.truncated_normal((n_features, n_labels)))
weights.initial_value

<tf.Tensor 'truncated_normal_9:0' shape=(120, 5) dtype=float32>

The weights are already randomized, so there isn't really a need to randominize the bias, so we can use the [tf.zeros](https://www.tensorflow.org/api_docs/python/tf/zeros) function to generate a tensor with zeros.

In [69]:
n_labels = 5
bias = tf.Variable(tf.zeros(n_labels))
bias.initial_value

<tf.Tensor 'zeros_2:0' shape=(5,) dtype=float32>

## TensorFlow Softmax

[softmax](https://www.tensorflow.org/api_docs/python/tf/nn/softmax) returns an array of prob values

In [73]:
# how i used softmax - fed softmax the logit_data directly
def run():
    output = None
    logit_data = [2.0, 1.0, 0.1]
    logits = tf.placeholder(tf.float32)
    
    # TODO: Calculate the softmax of the logits
    softmax = tf.nn.softmax([2.0, 1.0, 0.1])     
    
    with tf.Session() as sess:
        # TODO: Feed in the logit data
        output = sess.run(softmax)

    return output

run()

array([ 0.65900117,  0.24243298,  0.09856589], dtype=float32)

In [74]:
# udacity's version - they made a feed_dict
def run():
    output = None
    logit_data = [2.0, 1.0, 0.1]
    logits = tf.placeholder(tf.float32)

    softmax = tf.nn.softmax(logits)

    with tf.Session() as sess:
        output = sess.run(softmax, feed_dict={logits: logit_data})

    return output
run()

array([ 0.65900117,  0.24243298,  0.09856589], dtype=float32)

## One hot encoding

Use [LabelBinarizer](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelBinarizer.html) to turn labels into one hot encoded vectors:

In [75]:
import numpy as np
from sklearn import preprocessing

# Example labels
labels = np.array([1,5,3,2,1,4,2,1,3])

# Create the encoder
lb = preprocessing.LabelBinarizer()

# Here the encoder finds the classes and assigns one-hot vectors 
lb.fit(labels)

# And finally, transform the labels into one-hot encoded vectors
lb.transform(labels)

array([[1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0]])

## Cross entropy in Tensorflow

We use [tf.reduce_sum](https://www.tensorflow.org/api_docs/python/tf/reduce_sum) and [tf.log](https://www.tensorflow.org/api_docs/python/tf/log) to calculate cross-entropy in tensorflow

In [81]:
x = tf.reduce_sum([1, 2, 3, 4, 5])  # 15

In [83]:
x = tf.log(100.0)  # 4.60517

In [86]:
softmax_data = [0.7, 0.2, 0.1]
one_hot_data = [1.0, 0.0, 0.0]

softmax = tf.placeholder(tf.float32)
one_hot = tf.placeholder(tf.float32)

# TODO: Print cross entropy from session
cross_entropy = -tf.reduce_sum(tf.multiply(one_hot, tf.log(softmax)))
    
with tf.Session() as sess:
    output = sess.run(cross_entropy, feed_dict={softmax: softmax_data, one_hot: one_hot_data})
    print(output)

0.356675


## Normalize data

![](images/normalize_data.png)

Data should have zero mean and equal variance, else it can mess up all the maths. two main points:
- datasets can have some big and some vaules, like say house value is large (600,000), swimming pools is small (0 or 1), yet both data points are important in evaluating a house. So normalizing the data gives all the data a chance to make a difference in the output.
- math errors can happen when mixing large and small numbers, as the example below demonstrates.

In the example below, using a bg vs small initial number leads to a different result:

In [90]:
a, b = 1000000000, 1
for i in range(1000000):
    a = a + 1e-6
    b = b + 1e-6
a - 1000000000, b-1

(0.95367431640625, 0.9999999999177334)

There are many ways to initialize weights but a good rule of thumb is to start as a random distrubution around zero with a small standard deviation. A small sigma implies uncertainity, while large sigma implies certainity, so it's good to start with a small standard dev.

## Measuring Performance

